## 人脸与人脸关键点检测

在训练用于检测面部关键点的神经网络之后，你可以将此网络应用于包含人脸的*任何一个*图像。该神经网络需要一定大小的Tensor作为输入，因此，要检测任何一个人脸，你都首先必须进行一些预处理。

1. 使用人脸检测器检测图像中的所有人脸。在这个notebook中，我们将使用Haar级联检测器。
2. 对这些人脸图像进行预处理，使其成为灰度图像，并转换为你期望的输入尺寸的张量。这个步骤与你在Notebook 2中创建和应用的`data_transform` 类似，其作用是重新缩放、归一化，并将所有图像转换为Tensor，作为CNN的输入。
3. 使用已被训练的模型检测图像上的人脸关键点。

---

在下一个python单元格中，我们要加载项目此部分所需的库。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

#### 选择图像 

选择一张图像，执行人脸关键点检测。你可以在`images/`目录中选择任何一张人脸图像。

In [ ]:
import cv2
# load in color image for face detection
image = cv2.imread('images/obamas.jpg')

# switch red and blue color channels 
# --> by default OpenCV assumes BLUE comes first, not RED as in many images
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# plot the image
fig = plt.figure(figsize=(9,9))
plt.imshow(image)

## 检测该图像中的所有人脸

要想检测到所选图像中的所有人脸，接下来，你要用到的是OpenCV预先训练的一个Haar级联分类器，所有这些分类器都可以在`detector_architectures/`目录中找到。

在下面的代码中，我们要遍历原始图像中的每个人脸，并在原始图像的副本中的每个人脸上绘制一个红色正方形，而原始图像不需要修改。此外，你也可以 [新增一项眼睛检测 ](https://docs.opencv.org/3.4.1/d7/d8b/tutorial_py_face_detection.html) ，作为使用Haar检测器的一个可选练习。

下面是各种图像上的人脸检测示例。

<img src='images/haar_cascade_ex.png' width=80% height=80%/>

In [ ]:
# load in a haar cascade classifier for detecting frontal faces
face_cascade = cv2.CascadeClassifier('detector_architectures/haarcascade_frontalface_default.xml')

# run the detector
# the output here is an array of detections; the corners of each detection box
# if necessary, modify these parameters until you successfully identify every face in a given image
faces = face_cascade.detectMultiScale(image, 1.2, 2)

# make a copy of the original image to plot detections on
image_with_detections = image.copy()

# loop over the detected faces, mark the image where each face is found
for (x,y,w,h) in faces:
    # draw a rectangle around each detected face
    # you may also need to change the width of the rectangle drawn depending on image resolution
    cv2.rectangle(image_with_detections,(x,y),(x+w,y+h),(255,0,0),3) 

fig = plt.figure(figsize=(9,9))

plt.imshow(image_with_detections)

## 加载到已训练的模型中

有了一个可以使用的图像后（在这里，你可以选择`images/` 目录中的任何一张人脸图像），下一步是对该图像进行预处理并将其输入进CNN人脸关键点检测器。

首先，按文件名加载你选定的最佳模型。

In [ ]:
import torch
from models import Net

net = Net()

## TODO: load the best saved model parameters (by your path name)
## You'll need to un-comment the line below and add the correct name for *your* saved model
# net.load_state_dict(torch.load('saved_models/keypoints_model_1.pt'))

## print out your net and prepare it for testing (uncomment the line below)
# net.eval()

## 关键点检测

现在，我们需要再一次遍历图像中每个检测到的人脸，只是这一次，你需要将这些人脸转换为CNN可以接受的张量形式的输入图像。

### TODO: 将每个检测到的人脸转换为输入Tensor

你需要对每个检测到的人脸执行以下操作：
1. 将人脸从RGB图转换为灰度图
2. 把灰度图像归一化，使其颜色范围落在[0,1]范围，而不是[0,255]
3. 将检测到的人脸重新缩放为CNN的预期方形尺寸（我们建议为 224x224）
4. 将numpy图像变形为torch图像。

**提示**: Haar检测器检测到的人脸大小与神经网络训练过的人脸大小不同。如果你发现模型生成的关键点对给定的人脸来说，显得太小，请尝试在检测到的`roi`中添加一些填充，然后将其作为模型的输入。

你可能会发现，参看`data_load.py`中的转换代码对帮助执行这些处理步骤很有帮助。


### TODO: 检测并显示预测到的关键点

将每个人脸适当地转换为网络的输入Tensor之后，就可以将`net` 应用于每个人脸。输出应该是预测到的人脸关键点，这些关键点需要“非归一化”才能显示。你可能会发现，编写一个类似`show_keypoints`的辅助函数会很有帮助。最后，你会得到一张如下的图像，其中人脸关键点与每张人脸上的面部特征非常匹配：

<img src='images/michelle_detected.png' width=30% height=30%/>

In [ ]:
image_copy = np.copy(image)

# loop over the detected faces from your haar cascade
for (x,y,w,h) in faces:
    
    # Select the region of interest that is the face in the image 
    roi = image_copy[y:y+h, x:x+w]
    
    ## TODO: Convert the face region from RGB to grayscale

    ## TODO: Normalize the grayscale image so that its color range falls in [0,1] instead of [0,255]
    
    ## TODO: Rescale the detected face to be the expected square size for your CNN (224x224, suggested)
    
    ## TODO: Reshape the numpy image shape (H x W x C) into a torch image shape (C x H x W)
    
    ## TODO: Make facial keypoint predictions using your loaded, trained network 

    ## TODO: Display each detected face and the corresponding keypoints        
        
